In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data science libraries
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Others
from IPython.core.debugger import set_trace
from pathlib import Path

In [ ]:
from sklearn.preprocessing import StandardScaler

def scaling(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return scaler.transform(data)

In [ ]:
path = "/content/drive/MyDrive/Bearing/bearing_test_1/"

df = pd.read_csv(path + "bearing_concat.csv")

features = df.columns[:400]
target = 'target'
random_seed = 42

## Split the data into train and validation set
x_train, x_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.20, random_state=random_seed, shuffle=True)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

x_train = scaling(x_train)
x_test = scaling(x_test)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x) #여기서 뻑이났는데
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
model = build_model(input_shape,head_size=256, num_heads=4,ff_dim=4,num_transformer_blocks=4,mlp_units=[128],mlp_dropout=0.4,dropout=0.25,)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=["sparse_categorical_accuracy"],)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

hist = model.fit(x_train, y_train, validation_split=0.2, epochs=200, batch_size=128,callbacks=callbacks)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 400, 1)]     0           []                               
                                                                                                  
 layer_normalization_8 (LayerNo  (None, 400, 1)      2           ['input_2[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 400, 1)      7169        ['layer_normalization_8[0][0]',  
 eadAttention)                                                    'layer_normalization_8[0][0]']  
                                                                                            

In [ ]:
model.evaluate(x_test, y_test, verbose=1)

In [ ]:
from keras.models import load_model

model.save(path + 'transformer_encoder.h5')

In [ ]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)

print('')
print('loss_and_metrics : ' + str(loss_and_metrics))

75/75 [==============================] - 3s 41ms/step - loss: 0.3786 - sparse_categorical_accuracy: 0.8938

loss_and_metrics : [0.3785507082939148, 0.893750011920929]


In [ ]:
print('정답률 = ', loss_and_metrics[1],'loss=', loss_and_metrics[0])

정답률 =  0.893750011920929 loss= 0.3785507082939148


In [ ]:
hist = model

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train','test'], loc='upper left')
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train','test'], loc='upper left')
plt.show()

TypeError: ignored

In [ ]:
# 6. 모델 사용하기
xhat_idx = np.random.choice(x_test.shape[0], 5)
xhat = x_test[xhat_idx]

yhat = model.predict_classes(xhat)

for i in range(5):
    print('True : ' + str(argmax(y_test[xhat_idx[i]])) + ', Predict : ' + str(yhat[i]))

AttributeError: ignored